---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable's]`

Convert the energy supply and the energy supply per capita to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102), which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country.

*This function should return a DataFrame with 20 columns and 15 entries.*

In [3]:
def answer_one():
    import pandas as pd
    import numpy as np
    import re
    energy = (pd.read_excel("Energy Indicators.xls", skiprows=16,
                       parse_cols='C:F', skip_footer=39))
    energy.drop(energy.index[0], inplace=True) # drop the empty first record
    energy.rename(columns={'Unnamed: 0' : 'Country', 
                       'Energy Supply per capita' : 'Energy Supply per Capita',
                       'Renewable Electricity Production':'% Renewable'}, inplace=True)
    energy.replace("...", np.NaN, inplace=True)
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000 
    #remove citation # from country name as well as ( )
    energy['Country']= energy['Country'].apply(lambda x: re.sub("\d",'',x))
    energy['Country']= energy['Country'].apply(lambda x: re.sub(" +\(.*\)",'',x))
    names = {"Republic of Korea": "South Korea","United States of America": "United States",
            "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
            "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy.replace({'Country':names}, inplace=True)
    
    #reading gdp data
    gdp = pd.read_csv("world_bank.csv",  header=4)
    gdp.rename(columns={'Country Name':'Country'}, inplace=True)
    names= {"Korea, Rep.": "South Korea", 
            "Iran, Islamic Rep.": "Iran",
            "Hong Kong SAR, China": "Hong Kong"}
    gdp.replace({'Country':names}, inplace=True)
    scimen = pd.read_excel("scimagojr-3.xlsx")
    
    gdp_columns = ['Country', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    
    energy_gdp = pd.merge(energy, gdp[gdp_columns])
    alldata = pd.merge(energy_gdp, scimen[scimen['Rank'] < 16])
    alldata.set_index('Country', inplace = True)
    alldata.sort_values('Rank', ascending=True, inplace=True)
    correct_column_order = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 
                        'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009',
                        '2010', '2011', '2012', '2013', '2014', '2015']
    return alldata[correct_column_order]


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [2]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [59]:
def answer_two():
    import pandas as pd
    import re
    energy = (pd.read_excel("Energy Indicators.xls", skiprows=16,
                       parse_cols='C:F', skip_footer=39))
    energy.drop(energy.index[0], inplace=True)
    energy.rename(columns={'Unnamed: 0' : 'Country', 
                       'Energy Supply per capita' : 'Energy Supply per Capita',
                       'Renewable Electricity Production':'% Renewable'}, inplace=True)
    #remove citation # from country name as well as ( )
    energy['Country']= energy['Country'].apply(lambda x: re.sub("\d",'',x))
    energy['Country']= energy['Country'].apply(lambda x: re.sub(" +\(.*\)",'',x))
    #stripping white space at the end of Country column
    energy['Country']= energy['Country'].apply(lambda x: re.sub(" +$",'',x))
    names = {"Republic of Korea": "South Korea","United States of America": "United States",
            "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
            "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy.replace({'Country':names}, inplace=True)
    
    gdp = pd.read_csv("world_bank.csv",  header=4)
    gdp.rename(columns={'Country Name':'Country'}, inplace=True)
    names= {"Korea, Rep.": "South Korea", 
            "Iran, Islamic Rep.": "Iran",
            "Hong Kong SAR, China": "Hong Kong"}
    gdp.replace({'Country':names}, inplace=True)
    gdp['Country']= gdp['Country'].apply(lambda x: re.sub(" +$",'',x))
    scimen = pd.read_excel("scimagojr-3.xlsx")
    scimen['Country']= scimen['Country'].apply(lambda x: re.sub(" +$",'',x))
    c1 = set(energy['Country'])
    c2 = set(gdp['Country'])
    c3 = set(scimen['Country'])
    lost =  (c1 | c2 | c3) ^ (c1 & c2 & c3)
    return len(lost)

    # a different implmentation
    # all three data frames have Country column
    intersection = scimen.merge(energy).merge(gdp)
    intersection.set_index('Country', inplace = True)
    overlap = scimen.merge(energy, how='outer').merge(gdp, how='outer')
    return overlap['Country']
    overlap.set_index('Country', inplace = True)
    
    overlap.reset_index()
    intersection.reset_index()
    return len(overlap.index) - len(intersection.index)
answer_two()

157

### Question 3 (6.6%)
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [43]:
def answer_three():
    import pandas as pd
    import numpy as np
    
    columns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    #reading gdp data
    # gdp = pd.read_csv("world_bank.csv",  header=4)
    # gdp.rename(columns={'Country Name':'Country'}, inplace=True)
    '''names= {"Korea, Rep.": "South Korea", 
            "Iran, Islamic Rep.": "Iran",
            "Hong Kong SAR, China": "Hong Kong"}
    gdp.replace({'Country':names}, inplace=True)
    scimen = pd.read_excel("scimagojr-3.xlsx")
    # according to the Help forum, 2015 gdp is missing for Iran
    # in order to match instructor's result, one needs to replace NaN with 0
    # no sure why Iran will impact top 15, just park the code here for now
    # gdp.fillna(0)
    merged = gdp.merge(scimen).set_index('Country')
    '''
    Top15 = answer_one()
    #Top15 = Top15.fillna(0)
    def avgGDP(row):
        row['avgGDP'] = np.mean(row[columns])
        return row
    Top15 = (Top15.apply(avgGDP, axis=1).sort_values('avgGDP', ascending=False).head(15))
    return Top15['avgGDP']


Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [5]:
def answer_four():
    import pandas as pd
    import numpy as np
    columns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    Top15 = answer_one()
    Top15['avgGDP'] = Top15[columns].mean(axis=1)
    Top15 = Top15.sort_values('avgGDP', ascending=False)
    return Top15.iloc[5]['2015'] - Top15.iloc[5]['2006']
    


### Question 5 (6.6%)
What is the mean energy supply per capita?

*This function should return a single number.*

In [6]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()


### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [7]:
def answer_six():
    Top15 = answer_one()
    # return Top15[Top15["% Renewable"] == Top15["% Renewable"].max()]["% Renewable"]
    return tuple((Top15["% Renewable"].idxmax(), Top15["% Renewable"].max()))


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [8]:
def answer_seven():
    Top15 = answer_one()
    Top15['ratio'] = Top15['Self-citations']/Top15['Citations']    
    return tuple((Top15['ratio'].idxmax(), Top15['ratio'].max()))
   

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [9]:
def answer_eight():
    Top15 = answer_one()
    Top15['population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15.sort_values('population', inplace=True, ascending=False)
    return Top15.index[2]


### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita?

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita).*

In [51]:
def answer_nine():
    Top15 = answer_one()
    Top15['population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citation per Person'] = Top15['Citable documents'] / Top15['population']
    citation_capital = Top15['Citation per Person'].order()
    energy_capital = Top15['Energy Supply per Capita'].order()
    # return citation_capital.corr(energy_capital)
    return Top15['Citation per Person'].corr(Top15['Energy Supply per Capita'])


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: order is deprecated, use sort_values(...)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: order is deprecated, use sort_values(...)


0.79400104354429435

In [39]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [52]:
#plot9()

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [13]:
def answer_ten():
    Top15 = answer_one()
    median = Top15['% Renewable'].median()
    Top15['HighRenew'] = (Top15['% Renewable'] >= median)
    Top15['HighRenew'] = Top15['HighRenew'].apply(lambda x: 1 if x else 0)
    Top15.sort_values('Rank', inplace=True)
    return Top15['HighRenew']


### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [14]:
def answer_eleven():
    import pandas as pd
    import numpy as np
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15.reset_index(inplace=True)
    Top15['Continent'] = Top15['Country'].map(ContinentDict)
    #return Top15[['Continent','Country','Population']]
    return Top15.groupby('Continent')['Population'].agg({'size': np.size, 'sum': np.sum, 'mean': np.mean, 'std': np.std})


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [23]:
def answer_twelve():
    import pandas as pd
    import numpy as np
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15.reset_index(inplace=True)
    Top15['Continent'] = Top15['Country'].map(ContinentDict)
    Top15['bins for % Renewable'] = pd.cut(Top15['% Renewable'], 5)
    Top15 = Top15.groupby(['Continent', 'bins for % Renewable'])['Country'].agg(['count'])
    return Top15['count']


Continent      bins for % Renewable
Asia           (2.212, 15.753]         4
               (15.753, 29.227]        1
Australia      (2.212, 15.753]         1
Europe         (2.212, 15.753]         1
               (15.753, 29.227]        3
               (29.227, 42.701]        2
North America  (2.212, 15.753]         1
               (56.174, 69.648]        1
South America  (56.174, 69.648]        1
Name: count, dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas)

e.g. 12345678.90 -> 12,345,678.90

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [16]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15['PopEst'].apply(lambda x: "{:,}".format(x))


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [17]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [18]:
#plot_optional()